In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [30]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType


# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [31]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [32]:
# Get the data types of the columns. 
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [33]:
# Create a new DataFrame with an "price" column as a float datatype. 
df = df.withColumn("price", df["price"].cast("float"))
df = df.withColumn("bedrooms", df["bedrooms"].cast("float"))

df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: float (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [34]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [66]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""SELECT year(date) as year, bedrooms, round(avg(price), 2) AS average_price  FROM home_price WHERE bedrooms=4 group by 1, 2 ORDER BY 1 DESC""").show()


+----+--------+-------------+
|year|bedrooms|average_price|
+----+--------+-------------+
|2022|       4|    296363.88|
|2021|       4|    301819.44|
|2020|       4|    298353.78|
|2019|       4|     300263.7|
+----+--------+-------------+



In [90]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""select date_built, bedrooms, bathrooms, round(avg(price), 2) AS average_price from home_sales where bedrooms = 3 and bathrooms = 3 group by 1, 2, 3 ORDER BY 1 DESC""").show()


+----------+--------+---------+-------------+
|date_built|bedrooms|bathrooms|average_price|
+----------+--------+---------+-------------+
|      2017|     3.0|        3|    292676.79|
|      2016|     3.0|        3|    290555.07|
|      2015|     3.0|        3|     288770.3|
|      2014|     3.0|        3|    290852.27|
|      2013|     3.0|        3|    295962.27|
|      2012|     3.0|        3|    293683.19|
|      2011|     3.0|        3|    291117.47|
|      2010|     3.0|        3|    292859.62|
+----------+--------+---------+-------------+



In [105]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""select date_built, sqft_living, bedrooms, bathrooms, floors, round(avg(price), 2) AS average_price from home_sales 
where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
group by 1, 2, 3, 4, 5 ORDER BY 1 DESC""").show()



+----------+-----------+--------+---------+------+-------------+
|date_built|sqft_living|bedrooms|bathrooms|floors|average_price|
+----------+-----------+--------+---------+------+-------------+
|      2017|       2347|     3.0|        3|     2|     163645.0|
|      2017|       2465|     3.0|        3|     2|     308638.0|
|      2017|       2180|     3.0|        3|     2|     411691.0|
|      2017|       2208|     3.0|        3|     2|     152232.0|
|      2017|       2223|     3.0|        3|     2|     177718.0|
|      2017|       2211|     3.0|        3|     2|     224233.0|
|      2017|       2216|     3.0|        3|     2|     197183.0|
|      2017|       2047|     3.0|        3|     2|     337975.5|
|      2017|       2099|     3.0|        3|     2|     155474.0|
|      2017|       2221|     3.0|        3|     2|     133370.0|
|      2017|       2307|     3.0|        3|     2|     195236.0|
|      2017|       2135|     3.0|        3|     2|     229014.0|
|      2017|       2158| 

In [124]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""select view, round(avg(price), 2) AS average_price from home_sales where price >= 350000 group by 1 ORDER BY 1 DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.4684255123138428 seconds ---


In [110]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [111]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [125]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""select view, round(avg(price), 2) AS average_price from home_sales where price >= 350000 group by 1 ORDER BY 1 DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.33783459663391113 seconds ---


In [113]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [114]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('home_sales_partitioned')

In [115]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales')

In [126]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""select view, round(avg(price), 2) AS average_price from home_sales where price >= 350000 group by 1 ORDER BY 1 DESC""").show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.48078346252441406 seconds ---


In [117]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [118]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False